# Result_time_series_part

In [1]:
%cd "DoppelGANger"

C:\Users\zhang\DoppelGANger


In [2]:
import sys
import os
sys.path.append("..")
import matplotlib.pyplot as plt

from gan import output
sys.modules["output"] = output

import numpy as np
import pickle
import pandas as pd

from gan.doppelganger import DoppelGANger
from gan.load_data import load_data
from gan.network import DoppelGANgerGenerator, Discriminator, AttrDiscriminator
from gan.output import Output, OutputType, Normalization
import tensorflow as tf
from gan.network import DoppelGANgerGenerator, Discriminator, \
    RNNInitialStateType, AttrDiscriminator
from gan.util import add_gen_flag, normalize_per_sample

C:\Users\zhang\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zhang\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zhang\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zhang\AppData\Roaming\Python\Python35\site-

In [11]:
import seaborn as sns

## Load model

In [3]:
def normalize_attribute(data_att, data_att_outputs, data_att_min, data_att_max):
    
    data_att_norm = data_att
    total_dim = 0
    for output in data_att_outputs:
        if output.type_ == OutputType.CONTINUOUS:
            for _ in range(output.dim):
                data_att_norm[:, total_dim] = (data_att_norm[:, total_dim] - data_att_min[total_dim]) / (data_att_max[total_dim] - data_att_min[total_dim])
                if output.normalization == Normalization.MINUSONE_ONE:
                    data_att_norm[:, total_dim] = data_att_norm[:, total_dim] * 2.0 - 1.0

                total_dim += 1
        else:
            total_dim += output.dim


    return data_att_norm

attributes = np.load('europe_att_test.npy')
#attributes = np.load('seasia_att_test.npy')
train_sample_size = attributes.shape[0]

features = np.load('europe_feat_test.npy')
#features = np.load('seasia_feat_test.npy')

features = features.reshape(-1,119,1)

features_gt = features

gen_flags = np.ones((train_sample_size,119))


data_feature_outputs = [
	output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False)]
  #hourly solar radiation

data_attribute_outputs = [
	output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False),
  #lat
	output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False),
  #longi
  output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False),
  #height
  output.Output(type_=OutputType.CONTINUOUS,dim=2,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False),
  #norm vector (xy)
  #output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False),
  #glzr
  output.Output(type_=OutputType.CONTINUOUS,dim=32,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False),
  #latent_im
  output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False),
  #mon
  output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False),
  #inc
  output.Output(type_=OutputType.CONTINUOUS,dim=8,normalization=Normalization.ZERO_ONE,is_gen_flag=False)]
  #weather_stat
  #output.Output(type_=OutputType.DISCRETE,dim=656,normalization=None,is_gen_flag=False)]
  #shadow mask


#necessary inputs
data_all = features
data_attribut = attributes
data_gen_flag = gen_flags

sample_len = 17

# normalise data
(data_feature, data_attribute, data_attribute_outputs,
 real_attribute_mask) = normalize_per_sample(
        data_all, data_attribut, data_feature_outputs,
        data_attribute_outputs)

# add generation flag to features
data_feature, data_feature_outputs = add_gen_flag(
    data_feature, data_gen_flag, data_feature_outputs, sample_len)


data_attribute_min = np.amin(data_attribute, axis=0)
data_attribute_max = np.amax(data_attribute, axis=0)
np.save('europe_att_test_min.npy',data_attribute_min)
#np.save('seasia_att_test_min.npy',data_attribute_min)
np.save('europe_att_test_max.npy',data_attribute_max)
#np.save('seasia_att_test_max.npy',data_attribute_max)


data_attribute_normlized = normalize_attribute(data_attribute, data_attribute_outputs, data_attribute_min, data_attribute_max)


generator = DoppelGANgerGenerator(
    feed_back=True,
    noise=True,
    feature_outputs=data_feature_outputs,
    attribute_outputs=data_attribute_outputs,
    real_attribute_mask=real_attribute_mask,
    attribute_num_units =100,
    sample_len=sample_len,
    feature_num_units=100,
    feature_num_layers=2)

discriminator = Discriminator(num_units=100)
attr_discriminator = AttrDiscriminator(num_units=100)

checkpoint_dir = "solargan_training/results/checkpoint"
sample_dir = "solargan_training/results/sample"
time_path = "solargan_training/results/time/time.txt"
epoch = 200
batch_size = 100
g_lr = 0.0001
d_lr = 0.0001 
vis_freq = 1000
vis_num_sample = 1
d_rounds = 3
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe=10.0
attr_d_lr = 0.0001
g_attr_d_coe = 1.0
extra_checkpoint_freq = 1000
num_packing = 1


# config
run_config = tf.ConfigProto()
tf.reset_default_graph()


sess = tf.Session(config=run_config)


with sess.as_default() as sess:
    assert tf.get_default_session() is sess
    gan = DoppelGANger(
        sess=sess, 
        checkpoint_dir=checkpoint_dir,
        sample_dir=sample_dir,
        time_path=time_path,
        epoch=epoch,
        batch_size=batch_size,
        data_feature=data_feature,
        data_attribute=data_attribute,
        real_attribute_mask=real_attribute_mask,
        data_gen_flag=data_gen_flag,
        sample_len=sample_len,
        data_feature_outputs=data_feature_outputs,
        data_attribute_outputs=data_attribute_outputs,
        vis_freq=vis_freq,
        vis_num_sample=vis_num_sample,
        generator=generator,
        discriminator=discriminator,
        attr_discriminator=attr_discriminator,
        d_gp_coe=d_gp_coe,
        attr_d_gp_coe=attr_d_gp_coe,
        g_attr_d_coe=g_attr_d_coe,
        d_rounds=d_rounds,
        g_rounds=g_rounds,
g_lr=g_lr,
d_lr=d_lr,
attr_d_lr = attr_d_lr,
        num_packing=num_packing,
        extra_checkpoint_freq=extra_checkpoint_freq)
    
    #building & training
    gan.build()
    
# load the weights / change the path accordingly
    gan.load(checkpoint_dir)

Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/linear/dense/kernel:0' shape=(52, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/batch_norm/beta:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/batch_norm/gamma:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/batch_norm/moving_mean:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer0/batch_norm/moving_variance:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer1/linear/dense/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_addi/layer1/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_add

In [4]:
def renormalize_per_sample(data_feature, data_attribute, data_feature_outputs,
                           data_attribute_outputs, gen_flags,
                           num_real_attribute):
    attr_dim = 0
    for i in range(num_real_attribute):
        attr_dim += data_attribute_outputs[i].dim
    attr_dim_cp = attr_dim

    fea_dim = 0
    for output in data_feature_outputs:
        if output.type_ == OutputType.CONTINUOUS:
            for _ in range(output.dim):
                max_plus_min_d_2 = data_attribute[:, attr_dim]
                max_minus_min_d_2 = data_attribute[:, attr_dim + 1]
                attr_dim += 2

                max_ = max_plus_min_d_2 + max_minus_min_d_2
                min_ = np.zeros_like(max_plus_min_d_2 - max_minus_min_d_2)

                max_ = np.expand_dims(max_, axis=1)
                min_ = np.expand_dims(min_, axis=1)

                if output.normalization == Normalization.MINUSONE_ONE:
                    data_feature[:, :, fea_dim] = \
                        (data_feature[:, :, fea_dim] + 1.0) / 2.0

                data_feature[:, :, fea_dim] = \
                    data_feature[:, :, fea_dim] * (max_ - min_) + min_

                fea_dim += 1
        else:
            fea_dim += output.dim

    tmp_gen_flags = np.expand_dims(gen_flags, axis=2)
    data_feature = data_feature * tmp_gen_flags

    data_attribute = data_attribute[:, 0: attr_dim_cp]

    return data_feature, data_attribute

In [5]:
## list_index is just the WWR=0 case

def gen_annual (gt_feat_all,given_att_all,list_index,num_gen,location_index,renorm_factor):
    
    site_list = ['geneva','milan','paris','perlin','zurich']
    
    num_weeks = 52
    
    train_sample_size = given_att_all.shape[0]
    
    index = 3*list_index*num_weeks+int(location_index*train_sample_size/5)
    
    given_att = given_att_all[index:index+num_weeks,:-2]
    given_att_tiles=np.tile(given_att,(num_gen,1))
    
    length = int(gt_feat_all.shape[1] / sample_len)
    num_real_attribute=8
    
    addi_attribute_input_noise = gan.gen_attribute_input_noise(
                num_gen*num_weeks)
    feature_input_noise = gan.gen_feature_input_noise(
                num_gen*num_weeks, length)
    
    #print(feature_input_noise)
    input_data = gan.gen_feature_input_data_free(
                num_gen*num_weeks)
    # generate features, attributes and lengths
    gen_features, gen_attributes, gen_flags, lengths = gan.sample_from(
        None, addi_attribute_input_noise,
        feature_input_noise, input_data, given_attribute=given_att_tiles)


    #denormalise accordingly
    gen_features, gen_attributes = renormalize_per_sample(
        gen_features, gen_attributes, data_feature_outputs,
        data_attribute_outputs, gen_flags,
        num_real_attribute=num_real_attribute)
    
    
    
    gt = gt_feat_all[index:index+num_weeks,:]
    gen_features=gen_features*renorm_factor
    gen_features=gen_features.squeeze(2).reshape(num_gen,-1)
    
    gen_features = gen_features.transpose()
    
    gen_features = np.expand_dims(gen_features,0)
    gt = gt.reshape(1,-1,1)
    
    return gt, gen_features

In [6]:
def all_annual_batch (gt_feat_all,given_att_all,num,num_gen,location_index,renorm_factor):
    weather_data_array_morning_batch = np.zeros((364,4,num_gen))
    weather_data_array_morning_single = np.zeros((364,4,1))
    weather_data_array_evening_batch = np.zeros((364,3,num_gen))
    weather_data_array_evening_single = np.zeros((364,3,1))
    
    feat_gt_all = np.empty(shape=(0,8736,1))
    feat_gen_all = np.empty(shape=(0,8736,num_gen))
    
    for i in range(num):
        
        gt_i, gen_i = gen_annual(gt_feat_all,given_att_all,i,num_gen,4,renorm_factor)
        gt_i_daily = gt_i.reshape(364,-1,1)
        gen_i_daily = gen_i.reshape(364,-1,num_gen)
        
        gt_i_daily_complete = np.concatenate((weather_data_array_morning_single,gt_i_daily,weather_data_array_evening_single),axis=1)
        gen_i_daily_complete = np.concatenate((weather_data_array_morning_batch,gen_i_daily,weather_data_array_evening_batch),axis=1)
        
        gt_i_complete = np.expand_dims(gt_i_daily_complete.reshape(-1,1),0)
        gen_i_complete = np.expand_dims(gen_i_daily_complete.reshape(-1,num_gen),0)
        
        feat_gt_all = np.append(feat_gt_all,gt_i_complete).reshape(-1,8736,1)
        feat_gen_all = np.append(feat_gen_all,gen_i_complete).reshape(-1,8736,num_gen)
        
    return feat_gt_all, feat_gen_all

        
        
    

In [7]:
features_gt = np.load('europe_feat_test.npy')
#features_gt = np.load('seasia_feat_test.npy')

features_gt = features_gt.reshape(-1,119,1)

In [9]:
site_list = ['geneva','milan','paris','perlin','zurich']
#site_list =['hochiminh','jakarta','kualalumpur','pangkok','singapore']
for site_i in range(5):
    site = site_list[site_i]
    site_i_gt, site_i_gen = all_annual_batch(features_gt,data_attribute_normlized,1000,10,site_i,data_attribute_max[48])
    site_i_gt_file = site+'_gt_feat_test.npy'
    site_i_gen_file = site+'_gen_feat_test.npy' 
    
    np.save(site_i_gt_file,site_i_gt)
    np.save(site_i_gen_file,site_i_gen)
    